# Data Preprocessing


This file contains the code used for preprocessing the data. First, we get get all the data (extracted using python/cpp scripts) into a single CSV file. Then we begin preprocessing the data. We begin by cleaning the data by removing redundant features (highly correlated features). We also deal with the NaN entries.
Finally, we save the modified CSV file into another CSV file - `train2.csv` for training purposes.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

%matplotlib inline

In [ ]:
import os
os.chdir("drive/My Drive/Datasets")

In [ ]:
f_suffix = ["cookie", "fluency", "recall", "sentence"]

df1 = pd.read_csv("002-0.controlcookie.csv")
df2 = pd.read_csv("002-0.fcontrolcookie.csv")
df3 = pd.read_csv("control.csv")
df_train = df1.join(df2.set_index("File"), on = 'File', rsuffix = '_flucalc')
df_train = df_train.join(df3.set_index("File"), on = 'File')

for suf in f_suffix:
  df1 = pd.read_csv("001-0.dem" + suf + ".csv")
  df2 = pd.read_csv("001-0.fdem"+ suf + ".csv")
  df4 = pd.read_csv(suf+".csv")
  df3 = df1.join(df2.set_index("File"), on = 'File', rsuffix = '_flucalc')
  df3 = df3.join(df4.set_index("File"), on = 'File')
  df_train = pd.concat([df_train, df3])

In [ ]:
df_train

,File,Language,Corpus,Code,Age,Sex,Group,Race,SES,Role,Education,Custom_field,Duration_(sec),Total_Utts,MLU_Utts,MLU_Words,MLU_Morphemes,FREQ_types,FREQ_tokens,FREQ_TTR,Words_Min,Verbs_Utt,%_Word_Errors,Utt_Errors,density,%_Nouns,%_Plurals,%_Verbs,%_Aux,%_Mod,%_3S,%_13S,%_PAST,%_PASTP,%_PRESP,%_prep,%_adj,%_adv,%_conj,%_det,...,%_Block,#_PWR,%_PWR,#_PWR-RU,%_PWR-RU,#_WWR,%_WWR,#_mono-WWR,%_mono-WWR,#_WWR-RU,%_WWR-RU,#_mono-WWR-RU,%_mono-WWR-RU,Mean_RU,#_Phonological_fragment,%_Phonological_fragment,#_Phrase_repetitions,%_Phrase_repetitions,#_Word_revisions,%_Word_revisions,#_Phrase_revisions,%_Phrase_revisions,#_Pauses,%_Pauses,#_Filled_pauses,%_Filled_pauses,#_TD,%_TD,#_SLD,%_SLD,#_Total_(SLD+TD),%_Total_(SLD+TD),SLD_Ratio,Content_words_ratio,Function_words_ratio,Weighted_SLD,N,V,Q,S
0,002-0.cha,eng,Pitt,PAR,58;,female,Control,.,.,Participant,30,.,56,16,16,7.813,9.250,69,132,0.523,141.429,1.563,0.000,1,0.371,25.000,21.212,18.939,5.303,0.758,52.000,0.000,0.000,8.000,24.000,9.848,6.061,6.061,0.000,15.909,...,0.0,0.0,0.0,0.0,0.0,1.0,0.568,1.0,0.568,1.0,0.568,1.0,0.568,1.0,0.0,0.0,1.0,0.568,0.0,0.000,1.0,0.568,0.0,0.000,0.0,0.0,2.0,1.136,1.0,0.568,3.0,1.705,0.333,0.00,0.014,0.568,19.0,19.0,8,15
1,002-1.cha,eng,Pitt,PAR,59;,female,Control,.,.,Participant,30,.,65,15,14,6.500,7.714,50,102,0.490,94.154,1.067,0.000,2,0.353,29.412,20.000,15.686,7.843,0.000,68.750,0.000,0.000,0.000,50.000,3.922,6.863,3.922,0.000,14.706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,0.0,0.000,1.0,0.763,0.0,0.000,0.0,0.0,1.0,0.763,0.0,0.000,1.0,0.763,0.000,0.00,0.000,0.000,17.0,16.0,8,16
2,002-2.cha,eng,Pitt,PAR,60;,female,Control,.,.,Participant,30,.,62,14,14,9.857,12.071,68,145,0.469,140.323,1.571,0.000,1,0.400,21.379,29.032,15.862,6.207,0.690,60.870,0.000,0.000,4.348,30.435,9.655,6.207,8.276,2.759,14.483,...,0.0,0.0,0.0,0.0,0.0,1.0,0.513,1.0,0.513,1.0,0.513,1.0,0.513,1.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.000,1.0,0.513,1.0,0.513,1.000,0.00,0.012,0.513,NaN,NaN,NaN,NaN
3,002-3.cha,eng,Pitt,PAR,61;,female,Control,.,.,Participant,28,.,89,11,11,14.545,17.091,83,165,0.503,111.236,2.818,0.000,0,0.491,19.394,12.500,21.818,3.636,1.212,16.667,19.444,47.222,13.889,25.000,4.242,4.242,7.273,1.818,12.727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,1.0,0.461,2.0,0.922,0.0,0.000,0.0,0.000,0.0,0.0,3.0,1.382,0.0,0.000,3.0,1.382,0.000,0.00,0.000,0.000,18.0,28.0,14,22
4,006-2.cha,eng,Pitt,PAR,72;,male,Control,.,.,Participant,.,.,34,13,13,6.231,7.846,50,88,0.568,155.294,1.000,1.136,1,0.386,26.136,17.391,15.909,10.227,0.000,100.000,0.000,0.000,7.143,57.143,7.955,3.409,4.545,1.136,15.909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,1.0,0.885,0.0,0.000,0.0,0.000,0.0,0.0,1.0,0.885,1.0,0.885,2.0,1.770,0.500,0.00,0.000,1.770,18.0,18.0,8,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,703-0.cha,eng,Pitt,PAR,73;,female,ProbableAD,.,.,Participant,13,.,21,7,7,6.000,6.714,32,43,0.744,122.857,1.000,0.000,0,0.349,18.605,12.500,18.605,2.326,4.651,12.500,0.000,0.000,0.000,25.000,9.302,4.651,2.326,2.326,18.605,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.0,0.0,0.000,1.0,0.559,1.0,0.559,1.0,0.559,0.0,0.0,3.0,1.676,0.0,0.000,3.0,1.676,0.000,0.00,0.000,0.000,17.0,26.0,9,15
235,704-0.cha,eng,Pitt,PAR,50;,male,Other,.,.,Participant,23,.,65,6,6,14.833,17.667,58,92,0.630,84.923,3.167,0.000,0,0.457,17.391,37.500,20.652,3.261,1.087,31.579,0.000,0.000,5.263,21.053,7.609,2.174,5.435,2.174,9.783,...,0.0,0.0,0.0,0.0,0.0,1.0,0.383,1.0,0.383,2.0,0.766,2.0,0.766,2.0,0.0,0.0,1.0,0.383,2.0,0.766,1.0,0.383,2.0,0.766,0.0,0.0,6.0,2.299,2.0,0.766,8.0,3.065,0.250,0

In [ ]:
df_train.to_csv('train1.csv', index= False)

In [ ]:
df = pd.read_csv("train1.csv")

In [ ]:
df['S'].isna().sum()

229

In [ ]:
# 58 of the 108 columns were dropped - 
# 1. Manually judging which features would not be helpful or which have been already covered in some other column and hence are redundant.
# 2. Using correlation matrix, to figure out the correlation between 2 columns

cols = ['File', 'Language',	'Corpus',	'Code', 'Race',	'SES', 'Role',	'Education',	'Custom_field', 'Language_flucalc', 'Corpus_flucalc', 'Code_flucalc', 'Age(Month)', 'Sex_flucalc', 'Group_flucalc', 'Race_flucalc', 'SES_flucalc', 'Role_flucalc', 'Education_flucalc', 'Custom_field_flucalc', '#_Prolongation', '#_Broken_word', '#_Block', '#_PWR', '#_PWR-RU', '#_WWR', '#_mono-WWR', '#_WWR-RU', '#_mono-WWR-RU', '#_Phonological_fragment', '#_Phrase_repetitions', '#_Word_revisions', '#_Phrase_revisions', '#_Pauses', '#_Filled_pauses', '#_TD', '#_SLD', '#_Total_(SLD+TD)', '%_Total_(SLD+TD)']
cols += ['MLU_Words', 'Total_Utts', 'open_closed', '#open-class', '#closed-class', 'FREQ_types', 'FREQ_tokens', '%_SLD', 'Weighted_SLD', '%_WWR', '%_mono-WWR', '%_Block', '%_PWR', '%_PWR-RU', '%_WWR-RU', '%_mono-WWR-RU', '%_Broken_word']
cols += ['words_min', 'mor_Words']
df1 = df.drop(columns = cols)

In [ ]:
print(df1.columns)

Index(['Age', 'Sex', 'Group', 'Duration_(sec)', 'MLU_Utts', 'MLU_Morphemes',
       'FREQ_TTR', 'Words_Min', 'Verbs_Utt', '%_Word_Errors', 'Utt_Errors',
       'density', '%_Nouns', '%_Plurals', '%_Verbs', '%_Aux', '%_Mod', '%_3S',
       '%_13S', '%_PAST', '%_PASTP', '%_PRESP', '%_prep', '%_adj', '%_adv',
       '%_conj', '%_det', '%_pro', 'noun_verb', 'retracing', 'repetition',
       'mor_Utts', 'mor_syllables', 'syllables_min', '%_Prolongation',
       'Mean_RU', '%_Phonological_fragment', '%_Phrase_repetitions',
       '%_Word_revisions', '%_Phrase_revisions', '%_Pauses', '%_Filled_pauses',
       '%_TD', 'SLD_Ratio', 'Content_words_ratio', 'Function_words_ratio', 'N',
       'V', 'Q', 'S'],
      dtype='object')


In [ ]:
# Correlation matrix.

mask = np.zeros_like(df1.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.set_style('whitegrid')
plt.subplots(figsize = (15,12))
sns.heatmap(df1.corr(), 
            annot=False,
            cmap = 'RdBu',
            vmin = -1,
            vmax = 1,
            linewidths=.9, 
            linecolor='white',
            fmt='.2g',
            center = 0,
            square=True)

In [ ]:
corr = df1.corr()
for i in list(corr.columns):
    for j in list(corr.columns):
        if abs(corr.loc[i,j])>0.8 and corr.loc[i,j] != 1:
            print(i, '\t',j ,'\t', corr.loc[i,j])

MLU_Morphemes 	 Verbs_Utt 	 0.9179189852113331
Words_Min 	 syllables_min 	 0.9122940089968474
Verbs_Utt 	 MLU_Morphemes 	 0.9179189852113331
density 	 %_Nouns 	 -0.8387561684910999
%_Nouns 	 density 	 -0.8387561684910999
syllables_min 	 Words_Min 	 0.9122940089968474


In [ ]:
df1

,Age,Sex,Group,Duration_(sec),MLU_Utts,MLU_Morphemes,FREQ_TTR,Words_Min,Verbs_Utt,%_Word_Errors,Utt_Errors,density,%_Nouns,%_Plurals,%_Verbs,%_Aux,%_Mod,%_3S,%_13S,%_PAST,%_PASTP,%_PRESP,%_prep,%_adj,%_adv,%_conj,%_det,%_pro,noun_verb,retracing,repetition,mor_Utts,mor_syllables,syllables_min,%_Prolongation,Mean_RU,%_Phonological_fragment,%_Phrase_repetitions,%_Word_revisions,%_Phrase_revisions,%_Pauses,%_Filled_pauses,%_TD,SLD_Ratio,Content_words_ratio,Function_words_ratio,N,V,Q,S
0,58;,female,Control,56,16,9.250,0.523,141.429,1.563,0.000,1,0.371,25.000,21.212,18.939,5.303,0.758,52.000,0.000,0.000,8.000,24.000,9.848,6.061,6.061,0.000,15.909,5.303,1.320,1,2,18.0,176.0,178.983,0.000,1.0,0.0,0.568,0.000,0.568,0.000,0.0,1.136,0.333000,0.00,0.014,19.000000,19.000000,8.000000,15.000000
1,59;,female,Control,65,14,7.714,0.490,94.154,1.067,0.000,2,0.353,29.412,20.000,15.686,7.843,0.000,68.750,0.000,0.000,0.000,50.000,3.922,6.863,3.922,0.000,14.706,6.863,1.875,1,0,15.0,131.0,120.923,0.000,0.0,0.0,0.000,0.000,0.763,0.000,0.0,0.763,0.000000,0.00,0.000,17.000000,16.000000,8.000000,16.000000
2,60;,female,Control,62,14,12.071,0.469,140.323,1.571,0.000,1,0.400,21.379,29.032,15.862,6.207,0.690,60.870,0.000,0.000,4.348,30.435,9.655,6.207,8.276,2.759,14.483,6.897,1.348,0,1,16.0,195.0,172.059,0.000,1.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,1.000000,0.00,0.012,16.815287,17.732484,9.866242,14.993631
3,61;,female,Control,89,11,17.091,0.503,111.236,2.818,0.000,0,0.491,19.394,12.500,21.818,3.636,1.212,16.667,19.444,47.222,13.889,25.000,4.242,4.242,7.273,1.818,12.727,9.697,0.889,2,1,12.0,217.0,143.077,0.000,0.0,0.0,0.461,0.922,0.000,0.000,0.0,1.382,0.000000,0.00,0.000,18.000000,28.000000,14.000000,22.000000
4,72;,male,Control,34,13,7.846,0.568,155.294,1.000,1.136,1,0.386,26.136,17.391,15.909,10.227,0.000,100.000,0.000,0.000,7.143,57.143,7.955,3.409,4.545,1.136,15.909,6.818,1.643,1,0,14.0,113.0,183.243,0.885,0.0,0.0,0.000,0.885,0.000,0.000,0.0,0.885,0.500000,0.00,0.000,18.000000,18.000000,8.000000,13.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,65;,female,ProbableAD,22,6,6.833,0.811,100.909,0.667,0.000,0,0.351,32.432,25.000,10.811,5.405,0.000,25.000,50.000,50.000,25.000,0.000,18.919,8.108,0.000,0.000,16.216,8.108,3.000,0,0,9.0,68.0,120.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.000,0.224132,0.00,0.000,15.000000,16.000000,9.000000,10.000000
1271,73;,female,ProbableAD,21,7,6.714,0.744,122.857,1.000,0.000,0,0.349,18.605,12.500,18.605,2.326,4.651,12.500,0.000,0.000,0.000,25.000,9.302,4.651,2.326,2.326,18.605,6.977,1.000,0,0,24.0,179.0,108.485,0.000,0.0,0.0,0.000,0.559,0.559,0.559,0.0,1.676,0.000000,0.00,0.000,17.000000,26.000000,9.000000,15.000000
1273,71;,female,ProbableAD,48,6,6.000,0.694,45.000,1.500,0.000,0,0.333,27.778,0.000,25.000,0.000,0.000,44.444,0.000,11.111,0.000,0.000,2.778,2.778,2.778,0.000,16.667,8.333,1.111,0,1,16.0,86.0,55.484,0.000,1.0,0.0,0.000,0.000,0.000,2.326,0.0,2.326,0.500000,0.04,0.000,14.000000,13.000000,13.000000,15.000000
1274,74;,female,PossibleAD,26,6,8.667,0.717,106.154,1.167,0.000,0,0.391,28.261,23.077,15.217,2.174,0.000,28.571,0.000,28.571,0.000,14.286,13.043,4.348,4.348,0.000,19.565,6.522,1.857,1,0,7.0,60.0,124.138,1.667,0.0,0.0,0.000,0.000,1.667,1.667,0.0,3.333,0.333000,0.00,0.000,17.000000,14.000000,4.000000,13.000000


In [ ]:
# Certain rows in columns Q and S have a string written, starting with colon. We decided to delete those rows altogether, since Q and S
# are important features in making predictions

rows= []
df1['S'].value_counts()
for i in range(len(df1)):
  if str(df1.loc[i]['S'])[0] == ':':
    rows.append(i)
df1.drop(rows, axis = 0, inplace = True)

In [ ]:
df1['S'] = df1['S'].astype('float64')
df1['Q'] = df1['Q'].astype('float64')

In [ ]:
# Need to replace the NaNs in columns with the mean corresponding to their group

columns = ['Words_Min','noun_verb','syllables_min','SLD_Ratio','Function_words_ratio','N','V','S','Q']    # these columns have NaNs
for col in columns:
  for grp in ['Control', 'MCI', 'Memory', 'PossibleAD', 'ProbableAD', 'Vascular']:
    df1.loc[df1['Group'] == grp, col] = df1.loc[df1['Group'] == grp, col].replace(np.nan,df1.groupby('Group')[col].mean()[grp])

In [ ]:
print(df1.isna().sum().sum())        # Total number of NaN values in dataset currently

0


In [ ]:
# There are too many similar types of groups at present. We wish to combine the similar ones together.

df1.loc[df1["Group"] == "possibleAD", "Group"] = 'PossibleAD'
df1.loc[df1["Group"] == "Probable", "Group"] = 'ProbableAD'
df1.drop(df1[df1["Group"] == "Other"].index, inplace=True)
df1.drop(df1[df1["Group"] == "Dementia"].index, inplace=True)
df1.drop(df1[df1["Group"] == "."].index, inplace=True)

In [ ]:
# The age column at present, has a semi colon beside each age number. We need to remove the semicolon and convert it to 'int' 
# data type before feeding it to our model

for i in df1.index:
  try:
    df1['Age'][i] = str(df1.loc[i, 'Age'])[:2]
  except:
    pass

In [ ]:
df1['Age'] = df1['Age'].replace('.', np.nan)    
# Replacing the . entries with NaN, allows us to calculate the mean of the column. Later, we replace those NaN entries with the
# mean of the column.

df1['Age'] = df1['Age'].astype('float64')
df1['Age'] = df1['Age'].replace(np.nan, int(df1['Age'].mean()))

In [ ]:
# Encode Sex as - Male - 1 and Female - 0. We also replace some of the '.' entries in the column with the mode (Female)

df1['Sex'] = df1['Sex'].replace('.', 0)
df1['Sex'] = df1['Sex'].replace('female', 0)
df1['Sex'] = df1['Sex'].replace('male', 1)

In [ ]:
# Checking if any other column has a wierd character, that needs to be removed.
for col in df1.columns:
  print(col)
  print(df1[col].value_counts())
  print()

Age
69.0    205
73.0     60
78.0     57
64.0     54
65.0     51
75.0     50
71.0     46
68.0     43
67.0     42
66.0     40
70.0     40
74.0     39
77.0     38
79.0     38
72.0     38
63.0     34
57.0     32
62.0     27
59.0     25
80.0     25
85.0     25
56.0     23
83.0     23
61.0     21
58.0     19
81.0     18
76.0     18
84.0     17
60.0     17
87.0     11
55.0     11
50.0     10
51.0      8
88.0      8
86.0      8
89.0      7
54.0      7
82.0      7
52.0      6
49.0      5
53.0      5
90.0      4
47.0      2
48.0      2
46.0      1
Name: Age, dtype: int64

Sex
0    762
1    505
Name: Sex, dtype: int64

Group
ProbableAD    762
Control       243
MCI           162
PossibleAD     68
Vascular       20
Memory         12
Name: Group, dtype: int64

Duration_(sec)
36     34
19     30
21     27
24     27
22     26
       ..
125     1
109     1
103     1
100     1
0       1
Name: Duration_(sec), Length: 136, dtype: int64

MLU_Utts
6     247
7     106
9      91
8      89
5      89
11     56


In [ ]:
# Finally storing the preprocessed data in another csv file, so that we can directly load that file when required to train the model
df1.to_csv('train2.csv', index = False)   